**Project Overview**

This project simulates a competitive logistics ecosystem where six logistics companies (including our own) compete over the course of a year to fulfill transportation demands from ten independent client (production) companies. The simulation unfolds in monthly rounds. In each round, client companies decide how much to produce and which logistics company to contract, while logistics companies bid strategically to win contracts and maximize capacity utilization.

**Participants**

- **6 Logistics Companies**: Each logistics company is modeled as an intelligent agent. Five are independent competitors, and one is our own company. These companies operate under capacity constraints and pricing strategies, and they attempt to fulfill as many contracts as possible each month.

- **10 Client Companies**: These are production-based companies that manufacture between 100 and 400 tonnes of material per month. They must rely on logistics providers to transport their goods to market. If goods are not delivered in full, they are wasted.

**Objective**

The goal for logistics companies is to maximize the volume of material successfully transported, given their operational constraints and competition. For client companies, the goal is to maximize successful delivery of production output to the market. The market consumes all delivered goods — there is no demand constraint — but any undelivered material is lost.

**Logistics Capacity**

Each logistics company starts with a randomly assigned monthly carrying capacity between 200 and 1000 tonnes. This capacity cannot be exceeded. Logistics companies can bid to serve any number of clients, but must manage trade-offs between contract value and operational limits.

**Production Capacity**

Each client company has a fixed monthly production capacity between 100 and 400 tonnes. This does not vary throughout the year. Clients can produce up to their capacity, but only the volume that reaches the market via logistics partners is considered successful.

**Logistics Error Simulation**

Each logistics company experiences a random error percentage between 0% and 16% each month. This simulates real-world delivery unreliability — late arrivals, partial fulfillment, damage, etc. The error affects how much of the contracted shipment is actually delivered.

**Client–Logistics Relationship Dynamics**

Client companies track relationship scores for each logistics partner. This score changes each month based on that month’s delivery error:

| Error % | Relationship Change |
|---------|---------------------|
| 0       | +8                  |
| 1       | +7                  |
| 2       | +6                  |
| ...     | ...                 |
| 8       | ±0 (neutral)        |
| 9       | –1                  |
| 10      | –2                  |
| 11      | –3                  |
| 12      | –4                  |
| 13      | –5                  |
| 14      | –6                  |
| 15      | –7                  |
| 16      | –8                  |

Clients prefer to assign contracts to logistics companies with higher relationship scores, assuming competitive bids.

**Cost Structure of Logistics Companies**

Each logistics company has a unique and fixed cost structure (1–10 units) for three regions:
- In City
- Out of City
- Rural

These regional cost multipliers affect their bidding behavior and profitability. Each company’s regional cost preferences are private and do not change throughout the simulation.

**Regional Business Distribution of Clients**

Each client company operates in a fixed combination of:
- In City
- Out of City
- Rural

This distribution is constant for each client across all months and determines how much of their production must be delivered to each region. This regional breakdown affects logistics costs and bid attractiveness.

**Information Visibility and Privacy**

- Bids from logistics companies are **only visible to clients** and to the **logistics companies that submitted them**.
- No logistics company knows what others are doing unless they are directly competing for the same client.
- Clients compare all bids received and select logistics companies based on a weighted combination of cost, relationship score, and capacity.

**Simulation Mechanics**

Each monthly round proceeds in the following order:

1. **Client Production Planning**: Each client decides to produce up to their capacity.
2. **Logistics Bidding**: Each logistics company submits independent bids for one or more clients, factoring in region-specific costs and remaining capacity.
3. **Bid Evaluation by Clients**: Each client evaluates received bids and selects one or more logistics providers, weighted by relationship score and cost.
4. **Error Realization**: A random error percentage is applied to each winning logistics company for that month.
5. **Relationship Update**: Clients update relationship scores based on delivery error.
6. **Market Delivery Accounting**: Only the successfully delivered quantities are counted as reaching the market.

**Goals of the Simulation**

- To model intelligent bidding and adaptive logistics behavior
- To simulate long-term competitive market dynamics
- To balance operational constraints, reputation, and market pressure
- To investigate emergent strategies under uncertainty and competition

This simulation can serve as a foundation for testing reinforcement learning agents, game-theoretic decision making, or LLM-based reasoning in a multi-agent supply chain environment.


In [1]:
import random

class LogisticsCompany:
    def __init__(self, name, capacity, region_costs):
        self.name = name
        self.capacity = capacity  # total tonnes per month
        self.remaining_capacity = capacity  # updated each round
        self.region_costs = region_costs  # e.g., {"in_city": 4, "out_of_city": 7, "rural": 10}
        self.relationship_scores = {}  # client_name -> int
        self.error_rate = 0.0  # assigned each month (0.00 - 0.16)
        self.bids_this_month = []  # stores bids submitted this month

    def reset_month(self):
        self.remaining_capacity = self.capacity
        self.bids_this_month = []
        self.error_rate = round(random.uniform(0.0, 0.16), 2)

    def apply_error_to_volume(self, volume):
        return int(volume * (1.0 - self.error_rate))

    def update_relationship(self, client_name):
        error_percent = int(round(self.error_rate * 100))
        delta = 0
        if error_percent <= 2:
            delta = 8 - error_percent
        elif error_percent == 8:
            delta = 0
        elif error_percent > 8:
            delta = -1 * (error_percent - 8)

        self.relationship_scores[client_name] = self.relationship_scores.get(client_name, 0) + delta

    def register_relationship(self, client_name):
        if client_name not in self.relationship_scores:
            self.relationship_scores[client_name] = 0

    def place_bid(self, client, proposed_volume):
        if proposed_volume > self.remaining_capacity:
            return None  # Can't bid beyond current capacity

        cost = self.estimate_cost(client.region_split, proposed_volume)
        bid = {
            "client": client.name,
            "logistics_company": self.name,
            "proposed_volume": proposed_volume,
            "total_cost": cost,
            "unit_cost": round(cost / proposed_volume, 2),
            "relationship_score": self.relationship_scores.get(client.name, 0),
            "regions": client.region_split
        }

        self.bids_this_month.append(bid)
        return bid

    def estimate_cost(self, region_split, volume):
        total_cost = 0
        for region, percent in region_split.items():
            tonnes = volume * (percent / 100)
            cost_per_tonne = self.region_costs.get(region, 10)
            total_cost += tonnes * cost_per_tonne
        return int(round(total_cost))


In [2]:
import random

def create_random_logistics_company(name):
    capacity = random.randint(200, 1000)
    region_costs = {
        "in_city": random.randint(1, 10),
        "out_of_city": random.randint(1, 10),
        "rural": random.randint(1, 10)
    }
    return LogisticsCompany(name=name, capacity=capacity, region_costs=region_costs)

competitor_names = ["FreightNova", "ZipGo", "EcoHaul", "SkyLogix", "MegaMiles"]
logistics_companies = {}

for name in competitor_names:
    company = create_random_logistics_company(name)
    logistics_companies[name] = company

# Display the created companies
for name, company in logistics_companies.items():
    print(f"{name}: Capacity = {company.capacity}, Region Costs = {company.region_costs}")


FreightNova: Capacity = 468, Region Costs = {'in_city': 3, 'out_of_city': 6, 'rural': 7}
ZipGo: Capacity = 897, Region Costs = {'in_city': 10, 'out_of_city': 4, 'rural': 9}
EcoHaul: Capacity = 951, Region Costs = {'in_city': 5, 'out_of_city': 4, 'rural': 1}
SkyLogix: Capacity = 650, Region Costs = {'in_city': 8, 'out_of_city': 10, 'rural': 9}
MegaMiles: Capacity = 736, Region Costs = {'in_city': 5, 'out_of_city': 1, 'rural': 6}
